#  INIT

In [1]:
import sys
sys.path.append('../../')
sys.path.append('../infras/cellMix/')
sys.path.append('../infras/cytof_data/')
sys.path.append('../experiments/')
sys.path.append('../infras/dashboards')

In [2]:
import exploration_cytof_plots as cytof_plots

In [3]:
from cytof_cell_count_infra import CytofCellCountInfra
from cell_proportions_experiments import  CellProportionsExperiments

ccc= CytofCellCountInfra()

In [4]:
import pandas as pd
import numpy as np
from functools import partial
import multiprocessing

from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import seaborn as sns

import os

In [5]:
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins

from deconv_py.infras.data_factory import DataFactory
from deconv_py.infras.data_loader import DataLoader

from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation

from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot

from cellMix_coordinator import CellMixCoordinator

from scipy.stats import entropy
from functools import reduce

# calc cell proportions - intensities

## mixture & cell specific & proportions data

In [6]:
relvent_data = ['Intensity NOT_BCellmemory_01', 'Intensity NOT_BCellnaive_01', 'Intensity NOT_BCellplasma_01', 'Intensity NOT_CD4TCellmTregs_01',
 'Intensity NOT_CD4TCellnaive_01', 'Intensity NOT_CD4TCellnTregs_01', 'Intensity NOT_CD4TCellTcm_01', 'Intensity NOT_CD4TCellTem_01',
 'Intensity NOT_CD4TCellTemra_01', 'Intensity NOT_CD4TCellTh1_01', 'Intensity NOT_CD4TCellTh17_01', 'Intensity NOT_CD4TCellTh2_01',
 'Intensity NOT_CD8TCellnaive_01', 'Intensity NOT_CD8TCellTcm_01', 'Intensity NOT_CD8TCellTem_01', 'Intensity NOT_CD8TCellTemra_01',
 'Intensity NOT_DendriticCD1c_01', 'Intensity NOT_DendriticCD304_01', 'Intensity NOT_Erythrocytes_01', 'Intensity NOT_GranulocytesBasophil_01',
 'Intensity NOT_Granulocyteseosinophils_01', 'Intensity NOT_Granulocytesneutrophils_01', 'Intensity NOT_Monocytesclassical_01', 'Intensity NOT_Monocytesintermediate_01',
 'Intensity NOT_Monocytesnonclassical_01', 'Intensity NOT_NKCellsCD56bright_01', 'Intensity NOT_NKCellsCD56dim_01', 'Intensity NOT_Thrombocytes_01']

data_loader = DataLoader(mass_spec_path=os.path.abspath('../data/20190801_filtered_imputed_data.csv'),
                         protein_profile_path = os.path.abspath('../data/20150718_Cerberus_proteinGroups.txt'),as_csv = True)
ms = data_loader.get_mass_spec_data()

data_factory = DataFactory(data_loader)
profile_data,profile_data_relvent_data = data_factory.build_cell_specific_profile(auto_filter_by=False,relvent_data=relvent_data,relvent_columns=["Majority protein IDs"])
mixtures,mixtures_relvent_data = data_factory.build_mixture_data(relvent_data=[pb for pb in ms.columns if "_v" in pb],relvent_columns=["Majority protein IDs"],auto_filter_by=False,log2_transformation=True)

cell_proportions_df = pd.DataFrame(index = ['Intensity NOT_CD4TCellTcm_01','Intensity NOT_BCellmemory_01','Intensity NOT_Monocytesnonclassical_01'],
                                   columns = list(range(1,9)),
                                   data =np.array(((100,0,0),(0,100,0),(0,0,100),(33,33,33),(25,25,50),(25,50,25),(50,25,25),(47.5,47.5,5.0))).T)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: DtypeWarning: Columns (5,6,2764,2765,2773,2774,2775,2776) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## build A ,X and B 

In [7]:
index_func = lambda x:x.split(';')[0]
index_func = lambda x:x
_profile_data,_mixtures = PP_base.return_mutual_proteins_by_index(profile_data,mixtures,index_func=index_func)
A = _profile_data[profile_data_relvent_data]
B = _mixtures[mixtures_relvent_data]

X = cell_proportions_df

A = A.rename({f:f.split('Intensity ')[1] for f in A.columns},axis=1)

# models

### entropy base

In [ ]:
cell_prop_exp = CellProportionsExperiments()

In [9]:
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A.copy(deep=True), B.copy(deep=True))

_A_norm = _A.div(_A.sum(axis=1), axis=0)
gene_entropy = _A_norm.apply(lambda gene_dis: entropy(gene_dis), axis=1)

gene_entropy_trh = 0.001

cell_to_list_of_max_genes = {}
for protein,cell in  _A.idxmax(axis=1).to_dict().items() : 
    if cell in cell_to_list_of_max_genes.keys():
        cell_to_list_of_max_genes[cell].append(protein)
    else : 
        cell_to_list_of_max_genes[cell] = [] 

list_of_genes_list = []

for cell in _A.columns:
    n_genes_per_cell = 20
    cell_relvent_gene_entropy = gene_entropy[_A_norm[_A_norm[cell] > 0].index]
#     cell_relvent_gene_entropy = cell_relvent_gene_entropy.loc[cell_to_list_of_max_genes[cell]].dropna()
    #takse genes with zero entropy and the largest values
    zero_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[cell_relvent_gene_entropy == 0].index]
    best_zero_entropy_genes = zero_entropy_genes.nlargest(n_genes_per_cell).index
    
    best_genes_idx = best_zero_entropy_genes
    #how much genes we still need - 
    n_genes_per_cell = n_genes_per_cell - best_zero_entropy_genes.shape[0]
    trh = gene_entropy_trh
    while best_genes_idx.shape[0] <  n_genes_per_cell  :
        low_quantile_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[(cell_relvent_gene_entropy < cell_relvent_gene_entropy.quantile(trh)) & (cell_relvent_gene_entropy >0)].index]
        best_quantile_entropy_genes = low_quantile_entropy_genes.nlargest(n_genes_per_cell).index
        best_genes_idx = best_genes_idx.union(best_quantile_entropy_genes)
        trh=trh+gene_entropy_trh
        if trh > 1 :
            print(cell)
            break

    list_of_genes_list.append(best_genes_idx)
     
    list_of_genes_list.append(best_genes_idx)

genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

filt_A = _A.copy(deep=True).loc[genes_list_idx]
filt_B = _B.copy(deep=True).loc[genes_list_idx]

norm_filt_A = filt_A.div(filt_A.max(axis=1), axis=0)
norm_filt_B = filt_B.div(filt_A.max(axis=1), axis=0)

RuntimeError: No active exception to reraise

In [13]:
# _A.merge(gene_entropy.to_frame(),left_index=True,right_index=True)

,NOT_BCellmemory_01,NOT_BCellnaive_01,NOT_BCellplasma_01,NOT_CD4TCellmTregs_01,NOT_CD4TCellnaive_01,NOT_CD4TCellnTregs_01,NOT_CD4TCellTcm_01,NOT_CD4TCellTem_01,NOT_CD4TCellTemra_01,NOT_CD4TCellTh1_01,...,NOT_GranulocytesBasophil_01,NOT_Granulocyteseosinophils_01,NOT_Granulocytesneutrophils_01,NOT_Monocytesclassical_01,NOT_Monocytesintermediate_01,NOT_Monocytesnonclassical_01,NOT_NKCellsCD56bright_01,NOT_NKCellsCD56dim_01,NOT_Thrombocytes_01,0
major_protein_id,,,,,,,,,,,,,,,,,,,,,
A0A024RAC6;Q14241,316220000,39935000,6934100,244930000,210230000,73332000,247440000,214900000,191040000,40995000,...,133620000,66382000,53412000,63457000,180260000,165890000,285580000,6652400,0,3.006355
A0A087WVK0,0,0,0,0,0,0,0,0,0,0,...,83759000,0,0,27694000,42579000,0,0,0,0,1.173573
A0A087WXL6,700680000,219440000,0,889190000,973750000,309960000,1333900000,1891400000,924830000,248580000,...,1074200000,1675200000,288820000,465960000,2001100000,1290400000,1093200000,490430000,37127000,3.076460
A0A087WXP0,0,0,0,0,0,0,7085500,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
A0A087X0D5;P09668,433090000,468200000,50700000,501710000,59384000,58169000,448760000,444360000,321620000,319110000,...,2348300,13203000,158760000,1852300000,2903500000,1560300000,66009000,44738000,0,2.640603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y6X9;Q9Y6X9-2,109810000,8948900,4393100,79992000,231430000,37313000,177280000,346390000,78257000,48580000,...,25757000,59749000,0,0,34256000,123280000,206560000,5721700,0,2.828573
Q9Y6Y0,14113000,32325000,6346500,48008000,40203000,35063000,116420000,189060000,78766000,55101000,...,0,52280000,10449000,176480000,68608000,166830000,201110000,346610000,0,2.940080
Q9Y6Y9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2786000,21975000,88671000,0,0,0,0.601516


In [37]:
first_entropy = _A_norm.apply(lambda x:entropy(np.histogram(x,bins=15)[0]),axis=1) 
second_entropy = _A.apply(lambda x:entropy(np.histogram(x,bins=15)[0]),axis=1) 

In [38]:
print(first_entropy.corr(second_entropy))
print(first_entropy.corr(second_entropy,method="spearman"))

1.0
0.9999999999999998


#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(norm_filt_A,ax=ax1)
sns.heatmap(norm_filt_B,ax=ax2)
plt.show()
norm_filt_A.describe()
# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(norm_filt_A,norm_filt_B,None,True,True,True)
mass_results_not_none_df,cytof_not_none_count_df = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)


### entropy base - only largest among all cells

In [ ]:
def pp_entropy_largest_among_cells(A,B,n_genes_per_cell = 20,gene_entropy_trh=0.001,with_norm=True) : 
    _A_norm = _A.div(_A.sum(axis=1), axis=0)
    gene_entropy = _A_norm.apply(lambda gene_dis: entropy(gene_dis), axis=1)

    cell_to_list_of_max_genes = {}
    for protein,cell in  _A.idxmax(axis=1).to_dict().items() : 
        if cell in cell_to_list_of_max_genes.keys():
            cell_to_list_of_max_genes[cell].append(protein)
        else : 
            cell_to_list_of_max_genes[cell] = [] 

    list_of_genes_list = []

    for cell in _A.columns:
        _n_genes_per_cell = n_genes_per_cell
        cell_relvent_gene_entropy = gene_entropy[_A_norm[_A_norm[cell] > 0].index]
        cell_relvent_gene_entropy = cell_relvent_gene_entropy.loc[cell_to_list_of_max_genes[cell]].dropna()
        #takse genes with zero entropy and the largest values
        zero_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[cell_relvent_gene_entropy == 0].index]
        best_zero_entropy_genes = zero_entropy_genes.nlargest(_n_genes_per_cell).index

        best_genes_idx = best_zero_entropy_genes
        #how much genes we still need - 
        _n_genes_per_cell = _n_genes_per_cell - best_zero_entropy_genes.shape[0]
        trh = gene_entropy_trh
        
        while best_genes_idx.shape[0] <  _n_genes_per_cell  :
            low_quantile_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[(cell_relvent_gene_entropy < cell_relvent_gene_entropy.quantile(trh)) & (cell_relvent_gene_entropy >0)].index]
            best_quantile_entropy_genes = low_quantile_entropy_genes.nlargest(_n_genes_per_cell).index
            best_genes_idx = best_genes_idx.union(best_quantile_entropy_genes)
            trh=trh+gene_entropy_trh
            if trh > 1 :
                print(cell)
                break
        
        trh = gene_entropy_trh
        cell_relvent_gene_entropy = gene_entropy[_A_norm[_A_norm[cell] > 0].index]
        while best_genes_idx.shape[0] <  _n_genes_per_cell  :
            low_quantile_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[(cell_relvent_gene_entropy < cell_relvent_gene_entropy.quantile(trh)) & (cell_relvent_gene_entropy >0)].index]
            best_quantile_entropy_genes = low_quantile_entropy_genes.nlargest(_n_genes_per_cell).index
            best_genes_idx = best_genes_idx.union(best_quantile_entropy_genes)
            trh=trh+gene_entropy_trh
            if trh > 1 :
                print(cell)
                break

                
        list_of_genes_list.append(best_genes_idx)

    genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

    filt_A = _A.copy(deep=True).loc[genes_list_idx]
    filt_B = _B.copy(deep=True).loc[genes_list_idx]
    
    if not with_norm : 
        return filt_A,filt_B
    else : 
        norm_filt_A = filt_A.div(filt_A.max(axis=1), axis=0)
        norm_filt_B = filt_B.div(filt_A.max(axis=1), axis=0)
    return norm_filt_A,norm_filt_B

filt_A = A.copy(deep=True)
filt_A = filt_A[filt_A >filt_A.quantile(0.7)].fillna(0)
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A.copy(deep=True), B.copy(deep=True))
norm_filt_A,norm_filt_B = pp_entropy_largest_among_cells(filt_A, _B)

#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(norm_filt_A,ax=ax1)
sns.heatmap(norm_filt_B,ax=ax2)
plt.show()
norm_filt_A.describe()
pd.set_option('display.max_columns', None)

# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(norm_filt_A,norm_filt_B,None,True,True,True)
mass_results_not_none_df,cytof_not_none_count_df = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)


###  svm

In [ ]:
data_factory = DataFactory()
A_all_vs,B_all_vs = data_factory.load_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=True)

In [ ]:
cell_prop_exp = CellProportionsExperiments()

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC(multi_class="crammer_singer")

In [ ]:
samples = _A.copy().T

In [ ]:
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A_all_vs.copy(deep=True), B_all_vs.copy(deep=True))

list_of_genes_list = []

n_features = 20
samples = _A.copy().T

clf.fit(samples,[s.split("_0")[0] for s in samples.index])
# for cell in set(samples.index.map(lambda x:x.split("_0")[0])) :
#     y = pd.Series(index = samples.index,data = 0 )
#     y.loc[y.index.map(lambda x:cell in x)] = 1

#     model_res = clf.fit(samples,y)
#     list_of_genes_list.append(pd.Series(abs(model_res.coef_[0]), index=samples.columns).nlargest(n_features).index)

# genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

# filt_A = _A.copy(deep=True).loc[genes_list_idx]
# filt_B = _B.copy(deep=True).loc[genes_list_idx]

# norm_filt_A = filt_A.div(filt_A.max(axis=1), axis=0)
# norm_filt_B = filt_B.div(filt_A.max(axis=1), axis=0)

In [ ]:
clf.coef_.shape

In [ ]:
len(samples.index.to_list())

In [ ]:
norm_filt_A

#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(norm_filt_A,ax=ax1)
sns.heatmap(norm_filt_B,ax=ax2)
plt.show()
norm_filt_A.describe()
# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(norm_filt_A,norm_filt_B,None,True,True,True)
mass_results_not_none_df,_ = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)


### both

In [ ]:
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A.copy(deep=True), B.copy(deep=True))

_A_norm = _A.div(_A.sum(axis=1), axis=0)
gene_entropy = _A_norm.apply(lambda gene_dis: entropy(gene_dis), axis=1)

gene_entropy_trh = 0.001

list_of_genes_list = []

for cell in _A.columns:
    n_genes_per_cell = 150
    cell_relvent_gene_entropy = gene_entropy[_A_norm[_A_norm[cell] > 0].index]
    #takse genes with zero entropy and the largest values
    zero_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[cell_relvent_gene_entropy == 0].index]
    best_zero_entropy_genes = zero_entropy_genes.nlargest(n_genes_per_cell).index
    
    best_genes_idx = best_zero_entropy_genes
    #how much genes we still need - 
    n_genes_per_cell = n_genes_per_cell - best_zero_entropy_genes.shape[0]
    trh = gene_entropy_trh
    while best_genes_idx.shape[0] <  n_genes_per_cell  :
        low_quantile_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[(cell_relvent_gene_entropy < cell_relvent_gene_entropy.quantile(trh)) & (cell_relvent_gene_entropy >0)].index]
        best_quantile_entropy_genes = low_quantile_entropy_genes.nlargest(n_genes_per_cell).index
        best_genes_idx = best_genes_idx.union(best_quantile_entropy_genes)
        trh=trh+gene_entropy_trh

    list_of_genes_list.append(best_genes_idx)
     
    list_of_genes_list.append(best_genes_idx)

genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

_A, _B = _A.loc[genes_list_idx],_B.loc[genes_list_idx]

list_of_genes_list = []

n_features = 20
samples = _A.copy().T
for cell in samples.index:
    y = pd.Series(index = samples.index,data =0 )
    y.loc[cell] = 1
    
    model_res = clf.fit(samples,y)
    list_of_genes_list.append(pd.Series(abs(model_res.coef_[0]), index=samples.columns).nlargest(n_features).index)

genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

filt_A = _A.copy(deep=True).loc[genes_list_idx]
filt_B = _B.copy(deep=True).loc[genes_list_idx]

norm_filt_A = filt_A.div(filt_A.max(axis=1), axis=0)
norm_filt_B = filt_B.div(filt_A.max(axis=1), axis=0)


#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(norm_filt_A,ax=ax1)
sns.heatmap(norm_filt_B,ax=ax2)
plt.show()
norm_filt_A.describe()
# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(norm_filt_A,norm_filt_B,None,True,True,True)
mass_results_not_none_df,_ = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)


### entropy with const sum per cell   

In [ ]:
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A.copy(deep=True), B.copy(deep=True))

_A_norm = _A.div(_A.sum(axis=1), axis=0)
gene_entropy = _A_norm.apply(lambda gene_dis: entropy(gene_dis), axis=1)

gene_entropy_trh = 0.01

list_of_genes_list = []

percentage_of_amplitude = 0.0001
a_required_amplitude = percentage_of_amplitude * _A.sum().mean()
b_required_amplitude = percentage_of_amplitude * _B.sum().mean()
    
for cell in _A.columns:
    cell_relvent_gene_entropy = gene_entropy[_A_norm[_A_norm[cell] > 0].index]
    #takse genes with zero entropy and the largest values
    zero_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[cell_relvent_gene_entropy == 0].index]
    best_zero_entropy_genes = zero_entropy_genes.index
    
    best_genes_idx = best_zero_entropy_genes
    #how much genes we still need - 

    trh = gene_entropy_trh
    while ((_B.loc[best_genes_idx].sum().mean() <  b_required_amplitude) & (_A[cell].loc[best_genes_idx].sum() <  a_required_amplitude))  :
        low_quantile_entropy_genes = _A[cell].loc[cell_relvent_gene_entropy[(cell_relvent_gene_entropy < cell_relvent_gene_entropy.quantile(trh)) & (cell_relvent_gene_entropy >0)].index]
        best_quantile_entropy_genes = low_quantile_entropy_genes.index
        best_genes_idx = best_genes_idx.union(best_quantile_entropy_genes)
        trh=trh+gene_entropy_trh
    list_of_genes_list.append(best_genes_idx)

genes_list_idx = reduce(lambda x, y: x.union(y), list_of_genes_list)

filt_A = _A.copy(deep=True).loc[genes_list_idx]
filt_B = _B.copy(deep=True).loc[genes_list_idx]

norm_filt_A = filt_A.div(filt_A.max(axis=1), axis=0)
norm_filt_B = filt_B.div(filt_A.max(axis=1), axis=0)

#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(norm_filt_A,ax=ax1)
sns.heatmap(norm_filt_B,ax=ax2)
plt.show()
norm_filt_A.describe()
# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(norm_filt_A,norm_filt_B,None,True,True,True)
mass_results_not_none_df,cytof_not_none_count_df = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)


### only zero entropy

In [ ]:
_A, _B = PP_proteins.pp_clean_irrelevant_proteins(A.copy(deep=True), B.copy(deep=True))
_A_norm = _A.div(_A.sum(axis=1),axis=0)

only_zero_entropy = _A[_A_norm.apply(lambda gene_dis:entropy(gene_dis),axis=1)==0]

_filt_A = _A.loc[only_zero_entropy.index]
_filt_B =_B.loc[only_zero_entropy.index]

# non_zero_cells = _filt_A.loc[:, (_filt_A != 0).any(axis=0)].columns
# _filt_A = _filt_A[non_zero_cells.tolist()]

#### signature heatmap

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(18, 9))
plt.subplots_adjust(hspace=1)
sns.heatmap(_filt_A.loc[:, (_filt_A != 0).any(axis=0)],ax=ax1)
sns.heatmap(_filt_B,ax=ax2)
plt.show()
_filt_A.loc[:, (_filt_A != 0).any(axis=0)].describe()


# norm_filt_A[[col for col in norm_filt_A.columns if "8" in col]].describe()

# _B.loc["P13473"]/_A.loc["P13473"]["NOT_Monocytesclassical_01"]

#### results heatmap

In [ ]:
res_as_cytof,sample_over_cytof_count,mass_res = cell_prop_exp._calc_and_display_with_cellmix(_filt_A,_filt_B,None,True,True,True,normalize = True)
mass_results_not_none_df,cytof_not_none_count_df = ccc.return_mass_and_cytof_not_none_cells_counts(mass_res)

#### results scatters

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_all_on_one(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)

#### scatter per cell

In [ ]:
cytof_plots.plot_mass_to_cytof_scatter_per_cell(res_as_cytof,sample_over_cytof_count,mass_results_not_none_df)